# YOLOv8 Retraining & Hailo Compilation Guide

This notebook demonstrates how to retrain a YOLOv8 model using Ultralytics and compile it for Hailo devices using the Hailo Dataflow Compiler (DFC).

## Prerequisites

You need to download the following files from the [Hailo Developer Zone](https://hailo.ai/developer-zone/software-downloads/):
1.  **Hailo Dataflow Compiler (DFC)** Python wheel (e.g., `hailo_dataflow_compiler-*.whl`)
2.  **Hailo Model Zoo** Python wheel (e.g., `hailo_model_zoo-*.whl`)

**Note:** Ensure the DFC version supports the Python version of this Colab environment. Check `!python --version`.

In [ ]:
!python --version

## 1. Environment Setup

### Upload Hailo Wheels
Please upload the DFC and Model Zoo `.whl` files you downloaded.

In [ ]:
from google.colab import files

print("Please upload the Hailo DFC and Model Zoo wheels:")
uploaded = files.upload()

dfc_whl = [f for f in uploaded.keys() if "hailo_dataflow_compiler" in f]
hmz_whl = [f for f in uploaded.keys() if "hailo_model_zoo" in f]

if not dfc_whl or not hmz_whl:
    raise FileNotFoundError("Hailo DFC or Model Zoo wheels not uploaded. Please upload them to proceed.")
else:
    print(f"Found DFC: {dfc_whl[0]}")
    print(f"Found Model Zoo: {hmz_whl[0]}")

### Install Dependencies

In [ ]:
# Install Hailo wheels
if dfc_whl and hmz_whl:
    !pip install "{dfc_whl[0]}" "{hmz_whl[0]}"

# Install other dependencies
!pip install ultralytics kagglehub

# known PyTorch issue. The torch.package module has compatibility problems in certain PyTorch versions.
pip uninstall -y torch torchvision torchaudio
pip install torch==2.4.1 torchvision==0.19.1 torchaudio==2.4.1 --index-url https://download.pytorch.org/whl/cpu

## 2. Dataset Preparation

We will use the `barcode-detection` dataset from Kaggle as an example.

In [ ]:
import kagglehub
import shutil
import os

# Download latest version
path = kagglehub.dataset_download("kushagrapandya/barcode-detection")

print("Path to dataset files:", path)

# Move to a convenient location
dataset_dir = "/content/barcode-detect"
if not os.path.exists(dataset_dir):
    shutil.copytree(path, dataset_dir)
    print(f"Dataset moved to {dataset_dir}")
else:
    print(f"Dataset already exists at {dataset_dir}")

## 3. Retraining YOLOv8

We will retrain a YOLOv8s model. We use a small number of epochs for demonstration.

In [ ]:
from ultralytics import YOLO

# Load a model
model = YOLO('yolov8s.pt')  # load a pretrained model (recommended for training)

# Train the model
dataset_dir = "/content/barcode-detect"
results = model.train(data=f'{dataset_dir}/data.yaml', epochs=1, imgsz=640, batch=8, name='retrain_yolov8s')

## 4. Export to ONNX

Export the best model to ONNX format.

In [ ]:
success = model.export(format='onnx', opset=11)
print(f"Export success: {success}")

# Locate the exported ONNX file
import glob
onnx_files = glob.glob('runs/detect/retrain_yolov8s/weights/*.onnx')
if onnx_files:
    onnx_path = onnx_files[0]
    print(f"ONNX file found at: {onnx_path}")
    # Copy to a simple path
    !cp "{onnx_path}" /content/barcode-detection.onnx
else:
    print("Error: ONNX file not found.")

## 5. Compile to HEF

Now we use the Hailo Model Zoo to compile the ONNX model to a Hailo Executable Format (HEF) file.

First, we need to ensure we have the correct configuration files. Sometimes post-process configs are missing in the wheel installation, so we might need to fetch them.

In [ ]:
# Clone Hailo Model Zoo repo to get config files if needed
!git clone https://github.com/hailo-ai/hailo_model_zoo.git

# Setup environment variables if necessary or just point to the files in the cloned repo
import os
os.environ['HailsModelZoo'] = '/content/hailo_model_zoo'

### Run Compilation

We will use `hailomz compile`.
**Note**: Adjust `--hw-arch` if you are targeting a different device (e.g., `hailo8`, `hailo8l`, `hailo10h`). The Raspberry Pi 5 AI Kit uses `hailo8l`.

In [ ]:
!hailomz compile \
    --ckpt /content/barcode-detection.onnx \
    --calib-path /content/barcode-detect/valid/ \
    --yaml /content/hailo_model_zoo/hailo_model_zoo/cfg/networks/yolov8s.yaml \
    --classes 2 \
    --hw-arch hailo8l \
    --performance

## 6. Download HEF

The compilation should produce a `.hef` file. Download it to use on your Hailo device.

In [ ]:
hef_files = glob.glob('*.hef')
if hef_files:
    print(f"Found HEF files: {hef_files}")
    files.download(hef_files[0])
else:
    print("No HEF file generated. Check compilation output for errors.")